In [1]:
import pandas as pd

import scipy as sc
from scipy.io import loadmat

import numpy as np

from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

from matplotlib import pyplot as plt
%matplotlib inline

np.random.seed(123)

In [2]:
import util
from util import *

In [3]:
reload(util)

<module 'util' from 'util.pyc'>

## Prepare

1. Features: 
   - kernel of all features
   - kernel of each feature
   - gaussian, polynomial
   
2. Features preprocessing:
   - normalization of the features
   - spherical normalization of kernel matrices
   
3. levels of sparsity


# Read Data

### Breast Data

In [4]:
reload(util)

X_breast, y_breast = util.get_breast_data()
X_breast = util.preprocess_feats(X_breast)


N = int(X_breast.shape[0]*.75)

K_breast = util.get_kernels(X_breast, poly=True, feat_kernel=True, gauss=True,
                            max_poly_deg=2, max_gauss_width=4)
K_breast = util.preprocess_kernel(K_breast)

K, K_test = K_breast[:,:N,:N],K_breast[:,N:,:N]
y, y_test = y_breast[:N], y_breast[N:]

In [5]:
K.shape

(310, 426, 426)

# BEMKL

- Sparsity: #chosen kernels
- Running time: 
- Accuracy:

# Non-Sparse

In [6]:
import bemkl_binary

clf = bemkl_binary.BEMKL(sparse=False)

clf.bemkl_binary(K,y)

pred, prob = clf.predict(K_test)

print 'Accuracy:'
(pred==y_test).sum()*1./y_test.shape[0]

Accuracy:


0.986013986013986

In [7]:
e_nsparse = clf.mu_b_e[1:]

## Sparse

In [8]:
import bemkl_binary

clf = bemkl_binary.BEMKL(sparse=True)

clf.bemkl_binary(K,y)

pred, prob = clf.predict(K_test)

print 'Accuracy:'
(pred==y_test).sum()*1./y_test.shape[0]

Accuracy:


0.972027972027972

In [9]:
e_sparse = clf.mu_b_e[1:]